In [ ]:
#Date: 9/26/2022
#Author: Talia Kurtz

#Purpose of the code: This code is used to take the ARM data that has been resampled to hourly mean files for each year and fill the missing time stamps with NaNs to ensure that every file when\
#they are globbed together is the same size. This method uses the reindex_like function to do this. You will want to change the ds_time start/end to match the dataset. The frequency variable does not change 
#because every file is hourly data. The ARM data has a lot of variables that we are not interested in so to reduce the number of variables and the size of each file the variables that we are not interested
# in are dropped out of the data set. This code also pulls just the winter months as those are the only months we are interested in. 

#What is saved?
#This code saves the reduced, time filled in with nans files for each year to netCDF files to be used in another code that acts to NaN out bad data.

In [ ]:
#Imports
import xarray as xr
import numpy as np
import pandas as pd
import warnings
import re
from datetime import datetime
import xarray as xr
import glob

#Open the dataset you want to work with
PATH ='/Users/research/thesis_code/ARM_years/'
ds = xr.open_dataset(PATH+'arm_1hr_2022.nc')

In [ ]:
#Gnerate a dataset that has all the times for whatever you year you are looking at 
ds_time = xr.Dataset({'time':pd.date_range(start='2022-01-01 00:00:00',
                                            end='2022-06-05 23:00:00', freq='1H')})

In [ ]:
#Minus one to the dataarray will return the last values in the array so you can confirm that the ds_time ends where you want it too
ds_time['time'][-1]

In [ ]:
#This code snip goes through and compares the dataset you opened to the dataset you made in the previous two code blocks and 
#if there is a missing time it fills that time with a nan. This makes sure that you have no missing dates.
ds= ds.reindex_like(ds_time, method=None)

In [ ]:
#Grabbing only the Winter month data in this line.
ds_2 = ds.isel(time=ds.time.dt.month.isin([1,2,3,4,10,11,12]))

In [ ]:
#This code drops the unneeded variables so that we have a smaller file and less variables to work with.
dt = ds_2.drop(['qc_logger_temp','logger_temp','logger_volt','trh_err_code','dew_point_std','qc_dew_point_mean','dew_point_mean','qc_cmh_rh','cmh_rh','qc_cmh_vapor_pressure','cmh_vapor_pressure','qc_cmh_sat_vapor_pressure','cmh_sat_vapor_pressure','qc_cmh_dew_point','cmh_dew_point'])
dm = dt.drop(['qc_cmh_temp','cmh_temp','qc_pwd_cumul_rain','pwd_cumul_rain', 'vapor_pressure_mean', 'qc_vapor_pressure_mean','vapor_pressure_std', 'base_time'])

In [ ]:
#This line is for 2010 data since it has extra variables than the rest of the years 
#da = dm.drop(['qc_logger_volt','qc_time','qc_pws_cumul_snow','pws_cumul_snow','qc_pws_precip_rate_mean_1min','pws_precip_rate_mean_1min','qc_pws_pw_code_1hr','pws_pw_code_1hr','qc_pws_pw_code_15min','pws_pw_code_15min','qc_pws_pw_code_inst','pws_pw_code_inst','qc_pws_mean_vis_10min','pws_mean_vis_10min','qc_pws_mean_vis_1min','pws_mean_vis_1min','pws_err_code'])

In [ ]:
#Saves the new dataset to a .nc file
dm.to_netcdf('/Users/research/thesis_code/ARM_data_winter_months/ARM_2022_reduced.nc')